<a href="https://colab.research.google.com/github/aryanasadianuoit/adaptive_dml_test/blob/master/mobilenetv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade tensorflow-gpu
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from keras.datasets import cifar10
print(tf.__version__)
#Dataset First CIFAR-10
(train_images,train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = train_images /255.0
test_images = test_images /255.0
print(len(train_images))
print(len(test_images))


     |████████████████████████████████| 516.2MB 33kB/s 


Using TensorFlow backend.


2.2.0
170500096/170498071 [==============================] - 11s 0us/step
50000
10000


In [0]:
base_model_mobilenetv2 = keras.applications.MobileNetV2(input_shape= (32,32,3), include_top= False, weights= None)

**Defining custom Loss function**

In [0]:
import keras.backend as backend
def custom_loss(labels,logits_teacher, logits_student,middle_output_teacher, middle_output_student, temperature):

  #print("BEFORE", np.array(logits_teacher).shape)
  high_temp_outputs_teacher = backend.softmax(x= logits_teacher / temperature, axis= -1)
  high_temp_outputs_student = backend.softmax(x= logits_student / temperature, axis = -1)
  regular_prob_output = backend.softmax(x=logits_student, axis= -1)
 # print("Label Shape :  ", labels.shape, " Logit Shape : ",regular_prob_output.shape)
  #labels_tensor = tf.convert_to_tensor(labels)

 # print("Label type :  ", type(labels_tensor), " Logit type : ",type(regular_prob_output))
 # print("Part_1 shape", type(np.array(middle_output_student)))



#  print("DONE", high_temp_outputs_teacher.shape, "labels shape : ",labels.shape, "regular_prob_output", regular_prob_output.shape)
  #part_1 = keras.losses.sparse_categorical_crossentropy(y_true= labels,y_pred= regular_prob_output) 
  part_1 = keras.losses.SparseCategoricalCrossentropy()
  #part_1 = np.array(part_1)
 # assert part_1.shape == (2,)
  part_2 = keras.losses.kullback_leibler_divergence(y_true= high_temp_outputs_teacher, y_pred= high_temp_outputs_student) 
  part_3 = keras.losses.MSE(y_true= middle_output_teacher, y_pred= middle_output_student)
  #print("Part1 ==> ",type(part_1),"Part2 ==> ",type(part_2),"Part3 ==> ",type(part_3))
 # print("Shape Part1 ==> ",part_1.shape,"Part2 ==> ",part_2.shape,"Part3 ==> ",part_3.shape)

  
  #loss_value = part_1
  #+ part_2
 # + part_3
  return part_1
  , part_2
  , part_3

In [0]:
"""import keras.backend as backend
def custom_loss(labels,logits_teacher, logits_student,middle_output_teacher, middle_output_student, temperature):

  #print("BEFORE", np.array(logits_teacher).shape)
  high_temp_outputs_teacher = backend.softmax(x= logits_teacher / temperature)
  high_temp_outputs_student = backend.softmax(x= logits_student / temperature)
  print(high_temp_outputs_teacher)
  regular_prob_output = backend.softmax(x= logits_student)
  print("DONE", high_temp_outputs_teacher.shape, "labels shape : ",labels.shape, "regular_prob_output", regular_prob_output.shape)
  part_1 = keras.losses.sparse_categorical_crossentropy(y_true= labels,y_pred= regular_prob_output) 
  part_2 = keras.losses.kullback_leibler_divergence(y_true= high_temp_outputs_teacher, y_pred= high_temp_outputs_student) 
  part_3 = keras.losses.MSE(y_true= middle_output_teacher, y_pred= middle_output_student)
  print("Part1 ==> ",type(part_1),"Part2 ==> ",type(part_2),"Part3 ==> ",type(part_3))
  print("Shape Part1 ==> ",part_1.shape,"Part2 ==> ",part_2.shape,"Part3 ==> ",part_3.shape)

  
  #loss_value = part_1
  #+ part_2
 # + part_3
  return part_1
  , part_2
  , part_3"""

In [4]:
x = base_model_mobilenetv2.output
print(x.shape)
x = keras.layers.GlobalAveragePooling2D() (x)
print(x.shape)
x = keras.layers.Dense( units= 512, activation= keras.activations.relu) (x)
print(x.shape)
x = keras.layers.Dense( units= 256, activation= keras.activations.relu) (x)
print(x.shape)
x = keras.layers.Dense( units= 64, activation= keras.activations.relu) (x)
x = keras.layers.Dense( units= 10, activation= keras.activations.relu) (x)
print(x.shape)
predictions = keras.layers.Dense( units= 10, activation= keras.activations.softmax) (x)
print(predictions.shape)

(None, 1, 1, 1280)
(None, 1280)
(None, 512)
(None, 256)
(None, 10)
(None, 10)


In [11]:
model = keras.Model(inputs = base_model_mobilenetv2.input, outputs = predictions)
model2 = keras.Model(inputs = base_model_mobilenetv2.input, outputs = predictions)

#print(model2.layers[len(model2.layers)-1].get_weights())
#print("*****************")
#print((model2.layers[len(model2.layers)-2].output).shape)

complete_model = keras.Model(inputs=[model.input], outputs=[model.outputs, model2.outputs])

#for item in model2.layers[len(model2.layers)-2].output:
  #print(item)
#print("INJI +++++++++++++++++++>   ",model2.layers[len(model2.layers)-2].output.shape)
"""complete_model.compile(loss= custom_loss(labels= train_labels,logits_teacher= model2.layers[len(model2.layers)-2].output,
                                          logits_student= model.layers[len(model.layers)-2].output,
                                          middle_output_teacher= model2.layers[int(len(model2.layers)/2)].output,
                                          middle_output_student= model.layers[int(len(model.layers)/2)].output,temperature = 5),
                       metrics=["accuracy"], optimizer = keras.optimizers.Adam())"""


print("Shape of output : ",model.layers[len(model.layers)-2].output.shape," Shpe of Weights :",model.layers[len(model.layers)-2].get_weights)
#for item in model.layers[len(model.layers)-2].get_weights:
 # print(item)
complete_model.compile(loss= [custom_loss(labels= train_labels,logits_teacher= model2.layers[len(model2.layers)-2].output,
                                          logits_student= model.layers[len(model.layers)-2].output,
                                          middle_output_teacher= model2.layers[int(len(model2.layers)/2)].output,
                                          middle_output_student= model.layers[int(len(model.layers)/2)].output,temperature = 5),
                              custom_loss(labels= train_labels,logits_teacher= model.layers[len(model.layers)-2].output,
                                          logits_student= model2.layers[len(model2.layers)-2].output,
                                          middle_output_teacher= model.layers[int(len(model.layers)/2)].output,
                                          middle_output_student= model2.layers[int(len(model2.layers)/2)].output,temperature = 5)],
                       metrics=["accuracy"], optimizer = keras.optimizers.Adam())





#model.compile(loss="sparse_categorical_crossentropy",optimizer= keras.optimizers.Adam(),metrics =['accuracy'])
#model2.compile(loss="sparse_categorical_crossentropy",optimizer= keras.optimizers.Adam(),metrics =['accuracy'])

Shape of output :  (None, 10)  Shpe of Weights : <bound method Layer.get_weights of <tensorflow.python.keras.layers.core.Dense object at 0x7f830c033f60>>


In [12]:
complete_model.fit(x= train_images, y= train_labels, validation_data=(test_images, test_labels), epochs= 30, batch_size= 64)

Epoch 1/30
782/782 [==============================] - 15s 20ms/step - loss: 4.3468 - dense_4_loss: 2.1734 - dense_4_1_loss: 2.1734 - dense_4_accuracy: 0.1582 - dense_4_1_accuracy: 0.1582 - val_loss: 4.6621 - val_dense_4_loss: 2.3310 - val_dense_4_1_loss: 2.3310 - val_dense_4_accuracy: 0.1000 - val_dense_4_1_accuracy: 0.1000
Epoch 2/30
782/782 [==============================] - 15s 19ms/step - loss: 3.9484 - dense_4_loss: 1.9742 - dense_4_1_loss: 1.9742 - dense_4_accuracy: 0.2130 - dense_4_1_accuracy: 0.2130 - val_loss: 4.8082 - val_dense_4_loss: 2.4041 - val_dense_4_1_loss: 2.4041 - val_dense_4_accuracy: 0.1000 - val_dense_4_1_accuracy: 0.1000
Epoch 3/30
782/782 [==============================] - 15s 19ms/step - loss: 3.7963 - dense_4_loss: 1.8981 - dense_4_1_loss: 1.8981 - dense_4_accuracy: 0.2274 - dense_4_1_accuracy: 0.2274 - val_loss: 5.0784 - val_dense_4_loss: 2.5392 - val_dense_4_1_loss: 2.5392 - val_dense_4_accuracy: 0.1000 - val_dense_4_1_accuracy: 0.1000
Epoch 4/30
782/782 [==

In [0]:
num_epochs =10
model.fit(x= train_images, y= train_labels, validation_data=(test_images, test_labels), epochs= num_epochs, batch_size= 64)
model2.fit(x= train_images, y= train_labels, validation_data=(test_images, test_labels), epochs= num_epochs, batch_size= 64)

Epoch 1/10
782/782 [==============================] - 31s 40ms/step - loss: 1.8349 - accuracy: 0.2978 - val_loss: 2.3101 - val_accuracy: 0.1000
Epoch 2/10
782/782 [==============================] - 30s 38ms/step - loss: 1.5139 - accuracy: 0.4420 - val_loss: 2.3187 - val_accuracy: 0.1000
Epoch 3/10
782/782 [==============================] - 30s 38ms/step - loss: 1.3417 - accuracy: 0.5163 - val_loss: 2.3265 - val_accuracy: 0.1000
Epoch 4/10
782/782 [==============================] - 30s 38ms/step - loss: 1.2220 - accuracy: 0.5679 - val_loss: 2.3199 - val_accuracy: 0.1000
Epoch 5/10
782/782 [==============================] - 30s 39ms/step - loss: 1.1055 - accuracy: 0.6139 - val_loss: 2.2591 - val_accuracy: 0.1453
Epoch 6/10
782/782 [==============================] - 30s 39ms/step - loss: 1.0014 - accuracy: 0.6512 - val_loss: 1.8136 - val_accuracy: 0.3536
Epoch 7/10
782/782 [==============================] - 30s 39ms/step - loss: 0.9317 - accuracy: 0.6793 - val_loss: 1.3734 - val_accuracy:

In [0]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 33, 33, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 16, 16, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 16, 16, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

In [0]:
print(len(model.layers))

print(model.layers[79].name)
print(model.layers[79].output)
print(model.layers[79].get_weights())

160
block_8_project
Tensor("block_8_project/Identity:0", shape=(None, 2, 2, 64), dtype=float32)
[array([[[[-0.09529278,  0.1214966 , -0.08433251, ..., -0.06986708,
          -0.03856093,  0.02624267],
         [ 0.01554782, -0.02679724, -0.10795086, ..., -0.0810813 ,
           0.0654173 ,  0.03667847],
         [ 0.02402245,  0.20526394, -0.01564713, ..., -0.09118152,
          -0.05517881,  0.11630016],
         ...,
         [ 0.10672459,  0.19147508,  0.13557959, ...,  0.16707541,
          -0.14061964,  0.22443898],
         [ 0.03721062,  0.02795382,  0.01033777, ...,  0.10881364,
          -0.01892388,  0.10265918],
         [ 0.09094732, -0.08215877,  0.05763277, ...,  0.05380051,
           0.06636375,  0.09793741]]]], dtype=float32)]
